<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [ ]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2()

Run date time (IST): 2024-12-02 09:08:01
-------------------
qualified stocks: 164
with latest results: 154
still star stocks: 91


In [ ]:
# xsp
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XSP'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
6,AMBUJACEM,532.00,733.0,-13.36,35.28,11.59,0.09,37.78,30.64,12.45,XSP,BTT,0,0,H,CEMENT,0.0
3,ADANIPORTS,1201.90,1583.0,-13.16,4.97,3.66,1.00,31.71,32.30,37.38,XSP,NTT,1,1,M,MISC,NaN
128,ROSSARI,820.00,965.0,3.02,-31.32,83.54,0.31,17.68,91.30,22.34,XSP,NTT,1,1,M,CHEMICALS,-1.0
26,BOROLTD,459.85,500.5,18.82,NaN,85.37,0.34,8.84,8.84,53.13,XSP,ATH,1,0,M,GLASS,0.0
23,BIOCON,375.00,397.0,16.10,-41.82,96.34,2.70,5.87,27.09,57.35,XSP,NTT,1,1,H,PHARMA,0.0
153,UTIAMC,1298.55,1360.4,24.14,5.21,89.02,-0.05,4.76,4.76,64.07,XSP,ATH,1,1,M,FINANCE,0.0
89,JSWHL,14144.95,7487.0,76.64,NaN,100.00,0.00,-47.07,37.29,183.34,XSP,BTT,1,0,H,FINANCE,NaN


In [ ]:
# watchlist
df_tmp = df_prospects_features[df_prospects_features['InFolio'] == -1].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
0,3MINDIA,32245.30,50570.00,-3.90,-21.26,26.83,0.00,56.83,24.11,14.19,X40N,BTT,1,1,M,MISC,-1.0
149,TITAN,3249.00,4713.00,-6.64,-4.88,48.17,0.00,45.06,18.62,4.11,X40,BTT,1,1,H,JEWELLERY,-1.0
39,COLPAL,2890.00,3808.06,-3.92,23.20,16.46,0.01,31.77,31.77,27.87,X40,ATH,1,1,H,FMCG,-1.0
109,NESTLEIND,2242.80,2925.00,-9.86,-16.26,46.34,0.36,30.42,22.86,2.75,X40,BTT,0,0,H,FMCG,-1.0
13,ASTRAZEN,6598.95,8410.00,4.23,79.01,7.93,0.53,27.44,20.78,43.44,X40N,BTT,1,0,M,PHARMA,-1.0
119,PGHH,15900.00,19839.00,-2.23,-10.96,34.76,0.06,24.77,14.50,3.50,X40,BTT,0,0,H,FMCG,-1.0
159,WHIRLPOOL,1830.50,2270.00,2.49,14.33,4.27,-1.00,24.01,44.86,50.23,X40,NTT,1,0,H,DURABLES,-1.0
128,ROSSARI,820.00,965.00,3.02,-31.32,83.54,0.31,17.68,91.30,22.34,XSP,NTT,1,1,M,CHEMICALS,-1.0
108,NEOGEN,2149.00,2434.00,31.76,109.27,85.98,2.52,13.26,8.76,80.14,SR,BTT,1,0,L,CHEMICALS,-1.0


In [ ]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
149,TITAN,3249.00,4713.00,-6.64,-4.88,48.17,0.00,45.06,18.62,4.11,X40,BTT,1,1,H,JEWELLERY,-1.0
18,BAJAJ-AUTO,9078.80,12666.40,-5.74,61.19,10.37,0.50,39.52,39.52,50.79,X40,ATH,1,1,H,AUTO,NaN
39,COLPAL,2890.00,3808.06,-3.92,23.20,16.46,0.01,31.77,31.77,27.87,X40,ATH,1,1,H,FMCG,-1.0
61,GLAXO,2436.00,3182.00,-0.12,15.22,13.41,-0.34,30.62,25.10,47.77,X40,BTT,1,0,H,PHARMA,NaN
109,NESTLEIND,2242.80,2925.00,-9.86,-16.26,46.34,0.36,30.42,22.86,2.75,X40,BTT,0,0,H,FMCG,-1.0
119,PGHH,15900.00,19839.00,-2.23,-10.96,34.76,0.06,24.77,14.50,3.50,X40,BTT,0,0,H,FMCG,-1.0
159,WHIRLPOOL,1830.50,2270.00,2.49,14.33,4.27,-1.00,24.01,44.86,50.23,X40,NTT,1,0,H,DURABLES,-1.0
127,RELIANCE,1288.00,1590.07,-10.62,-50.97,25.61,-0.33,23.45,23.45,7.16,X40,ATH,1,1,H,REFINERIES,0.0
5,AKZOINDIA,3662.20,4433.10,21.67,14.46,53.05,-1.21,21.05,21.05,64.73,X40,ATH,1,1,H,PAINTS,NaN
77,ICICIPRULI,690.00,835.00,5.83,39.73,7.32,-1.38,21.01,14.64,44.07,X40,BTT,1,0,H,INSURANCE,0.0


In [ ]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
52,EQUITASBNK,63.00,113.10,-27.59,-31.79,6.71,1.07,79.52,79.52,1.19,X40N,ATH,1,0,M,BANKS,NaN
102,MOTILALOFS,919.95,1537.38,43.07,68.10,31.71,0.08,67.12,67.12,199.19,X40N,ATH,1,1,M,FINANCE,NaN
0,3MINDIA,32245.30,50570.00,-3.90,-21.26,26.83,0.00,56.83,24.11,14.19,X40N,BTT,1,1,M,MISC,-1.0
140,SYMPHONY,1325.60,2016.84,6.99,-20.68,4.88,-1.39,52.15,52.15,62.51,X40N,ATH,1,1,M,DURABLES,0.0
56,FINCABLES,1153.15,1657.02,-8.70,56.66,18.90,0.29,43.70,43.70,38.26,X40N,ATH,1,1,M,CABLES,NaN
73,HONAUT,41050.00,58518.52,-13.77,-15.63,1.83,0.53,42.55,42.55,17.14,X40N,ATH,1,1,M,ELECTRICAL,0.0
7,ANGELONE,2838.60,3791.17,6.96,-0.83,22.56,-2.02,33.56,33.56,38.37,X40N,ATH,1,1,M,FINANCE,NaN
13,ASTRAZEN,6598.95,8410.00,4.23,79.01,7.93,0.53,27.44,20.78,43.44,X40N,BTT,1,0,M,PHARMA,-1.0
156,VINATIORGA,1855.00,2325.26,1.70,12.15,42.07,-0.56,25.35,25.35,26.70,X40N,ATH,1,0,M,CHEMICALS,NaN
27,BOSCHLTD,34720.10,38795.60,6.91,2.16,20.12,-0.70,11.74,11.74,63.50,X40N,ATH,1,1,M,AUTO,NaN


In [ ]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,RSP,Today%,FTT%,ATH%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category,InFolio
19,BAJAJHIND,33.25,442.94,-7.64,NaN,50.00,-0.86,1232.15,1232.15,32.73,XR,ATH,1,0,M,SUGAR,0.0
8,APEX,235.82,849.16,0.35,NaN,49.39,0.03,260.09,260.09,24.02,AR,ATH,1,0,L,MISC,0.0
35,CENTRALBK,55.89,190.06,-8.38,-1.27,78.05,-0.05,240.06,240.06,19.04,XR,ATH,1,0,M,BANKS,NaN
121,PNB,104.74,232.89,-11.24,-41.07,75.00,-0.15,122.35,122.35,26.54,XR,ATH,1,1,M,BANKS,0.0
78,IGL,330.00,707.00,-28.88,-42.16,0.61,0.90,114.24,67.15,5.97,AR,BTT,1,1,L,GAS,NaN
42,DCBBANK,122.50,232.98,-2.78,NaN,79.88,-1.15,90.19,90.19,11.20,AR,ATH,1,1,L,BANKS,NaN
141,TASTYBITE,10787.95,20387.66,-9.02,81.14,8.54,0.00,88.99,88.99,15.43,XR,ATH,1,0,L,FMCG,0.0
111,NIACL,194.41,354.90,-17.62,41.34,47.26,0.00,82.55,82.55,13.98,XR,ATH,1,0,M,INSURANCE,0.0
82,INFIBEAM,27.09,49.37,-12.61,-21.41,39.63,0.26,82.24,82.24,34.98,AR,ATH,1,1,L,IT,NaN
87,J&KBANK,97.40,173.26,-16.03,15.38,60.98,-0.06,77.89,77.89,6.82,XR,ATH,1,1,M,BANKS,0.0


In [ ]:
df_prospects_features.columns

Index(['Symbol', 'Close', 'FTT', 'Dev%_200', 'Dev%_PE', 'RSP', 'Today%',
       'FTT%', 'ATH%', 'Gained%', 'Criteria', 'Strategy', 'LatestQtr',
       'StarStock', 'Conviction', 'Category', 'InFolio'],
      dtype='object')